In [34]:
import os
import pandas as pd

# Define the path to the folder
path = 'RMMS/'

# Get a list of all files in the folder
files = [f for f in os.listdir(path) if f.endswith('.lrps.htm')]

# Initialize an empty list to store DataFrames
dfs = []

# Loop through each file
for file in files:
    # Read HTML tables into a list of DataFrames
    tables = pd.read_html(os.path.join(path, file))
    
    # Combine all tables into a single DataFrame
    df = pd.concat(tables)
    
    # Remove the first row and rows above row 9
    df = df.iloc[9:].reset_index(drop=True)
    
    # Set the correct header using the values from row 9
    df.columns = df.iloc[0]
    
    # Drop the now redundant row 9
    df = df.iloc[1:].reset_index(drop=True)
    
    # Drop the first and last columns
    df = df.iloc[:, 1:-1]
    
    # Add a new column for road number
    road_number = file.split('.')[0]  # Extract road number from file name
    df['road_number'] = road_number
    
    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dfs, ignore_index=True)

# Save the combined DataFrame to a CSV file
combined_df.to_csv('combined_lrps.csv', index=False)


In [37]:
combined_df.shape
combined_df.to_csv('RMMS/combined_lrps.csv', index=False)

In [9]:
import pandas as pd

traffic_df = pd.read_csv('traffic.csv')
traffic_df.head()

,road,name,LRP,Offset,Chainage,LRP.1,Offset.1,Chainage.1,(Km),Heavy Truck,...,Car,Auto Rickshaw,Motor Cycle,Bi-Cycle,Cycle Rickshaw,Cart,Motorized,Non Motorized,Total AADT,(AADT)
0,N1,Jatrabari - Int.with Z1101 (Left) (Left),LRPS,0,0.000,LRPS,822,0.822,0.822,402.0,...,1851.0,2980.0,398.0,232.0,889.0,0.0,18236.0,1121.0,19357.0,19357.0
1,N1,Jatrabari - Int.with Z1101 (Left) (Right),LRPS,0,0.000,LRPS,822,0.822,0.822,660.0,...,2608.0,2508.0,436.0,213.0,1088.0,0.0,20236.0,1301.0,21537.0,21537.0
2,N1,Int.with Z1101 - Signboard (Left) R111 (Left),LRPS,822,0.822,LRPS,4175,4.175,3.353,660.0,...,2608.0,2508.0,436.0,213.0,1088.0,0.0,20236.0,1301.0,21537.0,21537.0
3,N1,Int.with Z1101 - Signboard (Left) R111 (Right),LRPS,822,0.822,LRPS,4175,4.175,3.353,402.0,...,1851.0,2980.0,398.0,232.0,889.0,0.0,18236.0,1121.0,19357.0,19357.0
4,N1,Signboard - Shimrail (Left)R110 (Left),LRPS,4175,4.175,LRPS,7181,7.181,3.006,91.0,...,1690.0,2266.0,1087.0,75.0,1198.0,0.0,16288.0,1273.0,17561.0,17561.0


In [12]:
traffic_df= traffic_df.rename(columns = { 'LRP':'LRP Start', 'Offset': 'Offset Start', 'Chainage':'Chainage Start', 'LRP.1':'LRP End', 'Offset.1': 'Offset End', 'Chainage.1':'Chainage End'})
traffic_df.head()

,road,name,LRP Start,Offset Start,Chainage Start,LRP End,Offset End,Chainage End,(Km),Heavy Truck,...,Car,Auto Rickshaw,Motor Cycle,Bi-Cycle,Cycle Rickshaw,Cart,Motorized,Non Motorized,Total AADT,(AADT)
0,N1,Jatrabari - Int.with Z1101 (Left) (Left),LRPS,0,0.000,LRPS,822,0.822,0.822,402.0,...,1851.0,2980.0,398.0,232.0,889.0,0.0,18236.0,1121.0,19357.0,19357.0
1,N1,Jatrabari - Int.with Z1101 (Left) (Right),LRPS,0,0.000,LRPS,822,0.822,0.822,660.0,...,2608.0,2508.0,436.0,213.0,1088.0,0.0,20236.0,1301.0,21537.0,21537.0
2,N1,Int.with Z1101 - Signboard (Left) R111 (Left),LRPS,822,0.822,LRPS,4175,4.175,3.353,660.0,...,2608.0,2508.0,436.0,213.0,1088.0,0.0,20236.0,1301.0,21537.0,21537.0
3,N1,Int.with Z1101 - Signboard (Left) R111 (Right),LRPS,822,0.822,LRPS,4175,4.175,3.353,402.0,...,1851.0,2980.0,398.0,232.0,889.0,0.0,18236.0,1121.0,19357.0,19357.0
4,N1,Signboard - Shimrail (Left)R110 (Left),LRPS,4175,4.175,LRPS,7181,7.181,3.006,91.0,...,1690.0,2266.0,1087.0,75.0,1198.0,0.0,16288.0,1273.0,17561.0,17561.0


In [18]:
# Gaat hier nu alleen op LRP nummer maar moet ook nog op weg nummer (bv N1 of Z8004), even uitzoeken hoe dit werkt. 
combined_lrps = pd.read_csv('combined_lrps.csv')

latlonload = pd.merge(traffic_df, combined_lrps, left_on='LRP Start', right_on='LRP No' )

In [20]:
latlonload.tail(10)

,road,name,LRP Start,Offset Start,Chainage Start,LRP End,Offset End,Chainage End,(Km),Heavy Truck,...,Non Motorized,Total AADT,(AADT),LRP No,Road Chainage,LRP TYPE,Description,Latitude Decimal,Longitued Decimal,road_number
3665444,Z8004,Kakaldi(int.with Z8005)-Int.with Z8203,LRP007c,0,7.61,LRP012,4800,16.8,9.19,NS,...,NS,NS,* NS,LRP007c,7.14,Others,Canal end,22.694083,90.1463886,Z8716
3665445,Z8004,Kakaldi(int.with Z8005)-Int.with Z8203,LRP007c,0,7.61,LRP012,4800,16.8,9.19,NS,...,NS,NS,* NS,LRP007c,7.67,Culvert,culvert,22.4294716,90.0509444,Z8740
3665446,Z8004,Kakaldi(int.with Z8005)-Int.with Z8203,LRP007c,0,7.61,LRP012,4800,16.8,9.19,NS,...,NS,NS,* NS,LRP007c,7.7,Culvert,Box Culvert,22.4294716,90.0509444,Z8740
3665447,Z8004,Kakaldi(int.with Z8005)-Int.with Z8203,LRP007c,0,7.61,LRP012,4800,16.8,9.19,NS,...,NS,NS,* NS,LRP007c,7.67,Culvert,culvert,22.4294716,90.0509444,Z8740
3665448,Z8004,Kakaldi(int.with Z8005)-Int.with Z8203,LRP007c,0,7.61,LRP012,4800,16.8,9.19,NS,...,NS,NS,* NS,LRP007c,7.7,Culvert,Box Culvert,22.4294716,90.0509444,Z8740
3665449,Z8004,Kakaldi(int.with Z8005)-Int.with Z8203,LRP007c,0,7.61,LRP012,4800,16.8,9.19,NS,...,NS,NS,* NS,LRP007c,7.67,Culvert,culvert,22.4294716,90.0509444,Z8740
3665450,Z8004,Kakaldi(int.with Z8005)-Int.with Z8203,LRP007c,0,7.61,LRP012,4800,16.8,9.19,NS,...,NS,NS,* NS,LRP007c,7.7,Culvert,Box Culvert,22.4294716,90.0509444,Z8740
3665451,Z8004,Kakaldi(int.with Z8005)-Int.with Z8203,LRP007c,0,7.61,LRP012,4800,16.8,9.19,NS,...,NS,NS,* NS,LRP007c,7.819,Culvert,Box Culvert,22.2380274,90.0687493,Z8814
3665452,Z8004,Kakaldi(int.with Z8005)-Int.with Z8203,LRP007c,0,7.61,LRP012,4800,16.8,9.19,NS,...,NS,NS,* NS,LRP007c,7.819,Culvert,Box Culvert,22.2380274,90.0687493,Z8814
3665453,Z8004,Kakaldi(int.with Z8005)-Int.with Z8203,LRP007c,0,7.61,LRP012,4800,16.8,9.19,NS,...,NS,NS,* NS,LRP007c,7.819,Culvert,Box Culvert,22.2380274,90.0687493,Z8814
